In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from scipy import signal as sig
from sklearn import preprocessing
from numpy import fft

In [ ]:
#initial constants
delta_t = 0.00025

nb_ptw = 2.048 #seconds

#voltage
V = 3.7

ptw_disc = int(nb_ptw/delta_t) #timesteps
msg_interval = 40000 #timesteps

to_uWh = (1000*1000*delta_t*V)/3600

In [ ]:
##CoAP
sweep_file1 = "../measurements/otii/sweep_coap_4.csv" #no psm
sweep_file2 = "../measurements/otii/sweep_coap_5.csv" #no psm
sweep_file3 = "../measurements/otii/sweep_coap_6.csv" #no psm

coap_data = []
coap_data.append(pd.read_csv(sweep_file1))
coap_data.append(pd.read_csv(sweep_file2))
coap_data.append(pd.read_csv(sweep_file3))

In [ ]:
## MQTT
sweep_file1 = "../measurements/otii/sweep_mqtt_2300_test1.csv"
sweep_file2 = "../measurements/otii/sweep_mqtt_2300_test2.csv"
sweep_file3 = "../measurements/otii/sweep_mqtt_2300_test3.csv"

mqtt_data = []
mqtt_data.append(pd.read_csv(sweep_file1))
mqtt_data.append(pd.read_csv(sweep_file2))
mqtt_data.append(pd.read_csv(sweep_file3))

In [ ]:
## MQTT long
sweep_file1 = "../measurements/otii/sweep_mqtt_4096_test1.csv"
sweep_file2 = "../measurements/otii/sweep_mqtt_4096_test2.csv"
sweep_file3 = "../measurements/otii/sweep_mqtt_4096_test3.csv"

mqtt_long_data = []
mqtt_long_data.append(pd.read_csv(sweep_file1))
mqtt_long_data.append(pd.read_csv(sweep_file2))
mqtt_long_data.append(pd.read_csv(sweep_file3))

In [ ]:
coap_energy = []
coap_current = []
for i in coap_data:
    coap_energy.append(i["Arc Main Energy (J)"])
    coap_current.append(i["Arc Main Current (A)"])

mqtt_energy = []
mqtt_current = []
for i in mqtt_data:
    mqtt_energy.append(i["Arc Main Energy (J)"])
    mqtt_current.append(i["Arc Main Current (A)"])
    
mqtt_long_energy = []
mqtt_long_current = []
for i in mqtt_long_data:
    mqtt_long_energy.append(i["Arc Main Energy (J)"])
    mqtt_long_current.append(i["Arc Main Current (A)"])


In [ ]:
plt.figure()
for i in mqtt_long_current:
    i.iloc[18500000:].plot()

In [ ]:
segments_coap = []

idx = 0
indexes_coap = []
for batch in coap_current:
    print("Batch: " + str(idx))
    curr_set = []
    i = 80000 
    index_list = []
    endpoint = (batch.index.size-10*ptw_disc)
    while(i < endpoint):
        curr_meas = batch.iloc[i]
        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i-ptw_disc):(i+3*ptw_disc)])
            i+=3*ptw_disc
            index_list.append(i)
        else:
            i += 1
    indexes_coap.append(index_list)
    segments_coap.append(curr_set)
    idx += 1

In [ ]:
plt.figure()
idx_seg = 0

coap_sums = np.zeros((len(segments_coap[0]), len(segments_coap)))
bytes_coap = range(0,5*len(segments_coap[0]), 5)


for i in segments_coap:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        coap_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_coap,sum_acc, label = str(idx_seg))
    
    idx_seg += 1

plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")
plt.legend()

In [ ]:
coap_stds = []
coap_avg = []

for i in coap_sums:
    coap_stds.append(np.std(i))
    coap_avg.append(np.mean(i))
    
np.mean(coap_stds)

In [ ]:
segments_mqtt = []
#before first tranmission - visually determined

#data end
mqtt_end = int(1.86*pow(10,7))

idx = 0

for batch in mqtt_current:
    print("Batch: " + str(idx))
    i = 200000
    curr_set = []
    
    while(i < mqtt_end):
        curr_meas = batch.iloc[i]

        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i-ptw_disc):(i+3*ptw_disc)])
            i += 4*ptw_disc
        else:
            i += 1
    segments_mqtt.append(curr_set)
    idx += 1

In [ ]:
plt.figure()
idx_seg = 0

mqtt_sums = np.zeros((len(segments_mqtt[0]), len(segments_mqtt)))
bytes_mqtt = range(0,5*len(segments_mqtt[0]), 5)

for i in segments_mqtt:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        mqtt_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt,sum_acc, label = str(idx_seg))
    idx_seg += 1

plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")
plt.legend()

In [ ]:
mqtt_stds = []
mqtt_avg = []

for i in mqtt_sums:
    mqtt_stds.append(np.std(i))
    mqtt_avg.append(np.mean(i))
    
np.mean(mqtt_stds)

In [ ]:
segments_mqtt_long = []

i = 200000

#observed
mqtt_long_end = int(1.656*pow(10,7))

idx = 0

for batch in mqtt_long_current:
    print("Batch: " + str(idx))
    i = 100000
    curr_set = []
    
    while(i < mqtt_long_end):
        curr_meas = batch.iloc[i]

        if(curr_meas > 0.04):    
            curr_set.append(batch.iloc[(i-ptw_disc):(i+3*ptw_disc)])
            i += 4*ptw_disc
        else:
            i += 1
    segments_mqtt_long.append(curr_set)
    idx += 1

In [ ]:
segments_mqtt_long[2][400].plot()

In [ ]:
plt.figure()
idx_seg = 0

mqtt_long_sums = np.zeros((len(segments_mqtt_long[0]), len(segments_mqtt_long)))
bytes_mqtt_long = range(0,10*len(segments_mqtt_long[0]), 10)


for i in segments_mqtt_long:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh 
        sum_acc.append(curr_sum)
        mqtt_long_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt_long, sum_acc, label = str(idx_seg))
    #plt.plot(sum_acc, label = str(idx_seg))
    idx_seg += 1
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")
plt.legend()

In [ ]:
mqtt_long_stds = []
mqtt_long_avg = []

for i in mqtt_long_sums:
    mqtt_long_stds.append(np.std(i))
    mqtt_long_avg.append(np.mean(i))

In [ ]:
coap_diff = np.diff(coap_avg)
mqtt_diff = np.diff(mqtt_avg)
mqtt_long_diff = np.diff(mqtt_long_avg)

coap_max_diff = np.max(coap_diff)
coap_max_loc = np.where(coap_diff == coap_max_diff)

mqtt_max_diff = np.max(mqtt_diff)
mqtt_max_loc = np.where(mqtt_diff == mqtt_max_diff)

mqtt_long_max_diff = np.max(mqtt_long_diff)
mqtt_long_max_loc = np.where(mqtt_long_diff == mqtt_long_max_diff)

print("CoAP avg diff: %5.6f" %(np.mean(coap_diff)))
print("CoAP max diff: %5.4f at %d bytes" %(coap_max_diff, coap_max_loc[0]*5))
print("MQTT avg diff: %5.6f" %(np.mean(mqtt_diff)))
print("MQTT max diff: %5.4f at %d bytes" %(mqtt_max_diff, mqtt_max_loc[0]*5))
print("MQTT long avg diff: %5.6f" %(np.mean(mqtt_long_diff)))
print("MQTT long max diff: %5.4f at %d bytes" %(mqtt_long_max_diff, mqtt_long_max_loc[0]*5))

## Boxplot showing the diff between the different segmetnt's power consumption
plt.figure()
plt.boxplot(coap_diff, positions = [1])
plt.boxplot(mqtt_diff, positions = [2])
plt.boxplot(mqtt_long_diff, positions = [3])
plt.xticks([1,2,3], ["CoAP 1430", "MQTT 2300","MQTT 4096"])

In [ ]:
coap_smooth = sig.savgol_filter(coap_avg,41,11)
mqtt_smooth = sig.savgol_filter(mqtt_avg,41,11)
mqtt_long_smooth = sig.savgol_filter(mqtt_long_avg,41,11)

In [ ]:
plt.figure()


plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

plt.title("Current consumption with respect to payload size")
#plt.plot(bytes_coap, coap_avg   ,       label = "CoAP average")
plt.plot(bytes_coap, coap_smooth, "--" ,label = "CoAP smoothed")
#plt.plot(bytes_mqtt, mqtt_avg   ,       label = "MQTT 2300 average")
plt.plot(bytes_mqtt, mqtt_smooth, "--" ,label = "MQTT 2300 smoothed")
#plt.plot(bytes_mqtt_long, mqtt_long_avg, label = "MQTT 4096 average")
plt.plot(bytes_mqtt_long, mqtt_long_smooth, label = "MQTT 4096 smoothed")

plt.legend()

In [ ]:
coap_avg[0]

In [ ]:
segments_coap[0][3].plot()